## Generic way to create a heatmap using weighted points



In [1]:
# Dependencies and Setup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

pd.options.display.float_format = '{0:,.3f}'.format
pd.set_option('display.max_columns', 500)

# File to Load 
geo_data_to_load = "Police_Geo.csv"

geo_data = pd.read_csv(geo_data_to_load)


In [2]:
# Round Geo Data to .001

geo_data.head()

def q(val):
    return round(val,3)

geo_data["long"] = geo_data["X"].map(q)
geo_data["lat"] = geo_data["Y"].map(q)
geo_data.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,long,lat
0,180000417,ASSAULT,BATTERY,Sunday,12/31/2017,23:58,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403,37.775,"(37.775420706711, -122.40340479147905)",18000041704134,-122.403,37.775
1,180000069,ASSAULT,AGGRAVATED ASSAULT WITH A KNIFE,Sunday,12/31/2017,23:56,TENDERLOIN,"ARREST, BOOKED",500 Block of JONES ST,-122.413,37.786,"(37.78627745916602, -122.41299907500884)",18000006904012,-122.413,37.786
2,180000069,BURGLARY,"BURGLARY OF APARTMENT HOUSE, FORCIBLE ENTRY",Sunday,12/31/2017,23:56,TENDERLOIN,"ARREST, BOOKED",500 Block of JONES ST,-122.413,37.786,"(37.78627745916602, -122.41299907500884)",18000006905011,-122.413,37.786
3,180000069,WEAPON LAWS,EXHIBITING DEADLY WEAPON IN A THREATING MANNER,Sunday,12/31/2017,23:56,TENDERLOIN,"ARREST, BOOKED",500 Block of JONES ST,-122.413,37.786,"(37.78627745916602, -122.41299907500884)",18000006912030,-122.413,37.786
4,186005077,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Sunday,12/31/2017,23:55,MISSION,NONE,500 Block of VALENCIA ST,-122.422,37.764,"(37.76408889445322, -122.42187648849193)",18600507706372,-122.422,37.764


In [3]:
# Group incidents by rounded coordinates

geo_slice_group=geo_data.groupby(["lat","long"]).agg(
    {"IncidntNum" : "count",
    "lat":"max",
    "long":"max"})
geo_slice_group.head()

IncidntNum    lat     long
lat    long                                
37.708 -122.486           8 37.708 -122.486
       -122.466           6 37.708 -122.466
       -122.461           2 37.708 -122.461
       -122.460           3 37.708 -122.460
       -122.459           1 37.708 -122.459

In [4]:
# Put rounded coordinates into a list of tuples

def to_gis (x, y):
    yy = ()
    yy = yy + (x,)
    yy = yy + (y,)
    return tuple(yy)

point_list = []

point_list = geo_slice_group[['lat','long']].apply(tuple, axis=1)

In [5]:
# Determine weights for each point

weights = list(geo_slice_group["IncidntNum"])

In [6]:
# How many points do we have?

print (len(point_list))

7983


In [7]:
# Set up the Google Maps API Call
import requests
import json

# Google developer API key
from config import gkey
import gmaps

gmaps.configure(api_key=gkey)

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [8]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations=point_list,weights=weights,point_radius=20))
fig

Figure(layout=FigureLayout(height='420px'))